# Data upload + API

In [1]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("literature/book.pdf")
pages = loader.load_and_split()

In [2]:
sources = pages

In [21]:
#print(pages[10])

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [34]:
embeddings = OpenAIEmbeddings()
# embeddings = embedding_functions.OpenAIEmbeddingFunction(
#                 model_name="text-embedding-ada-002"
#             )

In [35]:
source_chunks = []
splitter = CharacterTextSplitter(separator=" ", chunk_size=2024, chunk_overlap=0)
for source in sources:
    for chunk in splitter.split_text(source.page_content):
        source_chunks.append(Document(page_content=chunk, metadata=source.metadata))

In [36]:
import chromadb
client = chromadb.PersistentClient(path="data")

In [37]:
search_index = Chroma.from_documents(source_chunks, embedding=embeddings, client=client)

# Load 

In [ ]:
#search_index = Chroma.load_local("data")
# load from disk
# Note: The following code is demonstrating how to load the Chroma database from disk.
#search_index = Chroma(persist_directory="data")

In [42]:
#docs = await search_index.asimilarity_search('Description of Nusya')
#docs

In [40]:
retriever = search_index.as_retriever()

In [9]:
#retriever = search_index.as_retriever()

# Similarytiy with pages

In [61]:
docs = search_index.similarity_search("Nusya expreses her emotions", k=5)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:500])

71: The fact that she now approaches —in the same gliding walk instilled by Lviv’s most 
glorious tuteurs (only the legs, my lady, we’re moving only our legs!) beside Nusya, his 
regular courier, and carries toward him that serene, unattainable smile of hers like a 
discrete source of light in the November cityscape —is equivalent to the heavens collapsing 
in pieces onto the earth below —he wouldn’t blink an eye if they did collapse. Snow falls and 
carries the smell of her hair, the dizzyingly ten
325: think we are, these bastards? Do they think they’ll get away with this? I opened my 
shoulders wide and stood up tall facing a real , worthwhile enemy, one with whom you want  
to cross swords —while Lolly, by contrast, remained amazingly composed the whole time, 
as if she weren’t really surprised to hear my news (I’ve always suspected that she disliked 
Yulichka). She asked succinct, businesslike questions and generally appeared quite calm —
as if all the blows that came this evening

# Chat with AI about book

In [59]:
question = "How exactly Nusya challenges traditional gender roles? Provide points from the text."

In [60]:
template = """You are a professor expert in postcolonial theory and literary analysis in the context of the Russian-Ukrainian war. 
I haven't read Zabuzhko's book so I will need you to provide detailed summaries and analyses to help me understand the role of female memory spheres in the novel, 
especially in relation to the construction of Ukrainian cultural identity. You provide me long, detailed answers supported by specific examples from the text.
Please address the following points through both a postcolonial and a feminist lens:
- Elaborate on concepts of postcolonial theory and how they apply to Ukrainian cultural identity.
- Identify and elaborate on recurring themes, symbols, and narrative techniques in the novel.
- Analyze the interactions between the war and Ukrainian cultural identity from a female perspective.
Provide open long, detailed answers, with examples from the text.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)
result = qa_chain({"query": question})
result["result"]

'In the novel, Nusya challenges traditional gender roles in several ways. Firstly, she is depicted as a strong and independent woman who actively participates in the resistance movement. Despite being seen as a "wench" by some of her male comrades, Nusya proves herself to be a valuable member of the insurgency, serving as Adrian\'s courier for many years. This challenges the notion that women should only be confined to domestic roles and highlights their capability to contribute to political and military endeavors.\n\nAdditionally, Nusya\'s behavior and mannerisms defy societal expectations of femininity. She is described as having a "coy, kittenish" demeanor, which is attributed to her conservative Polish upbringing and her flirtatious nature. However, the novel raises the question of whether Nusya\'s behavior is a result of her own agency or a product of societal conditioning. Adrian initially dismisses her actions as a consequence of her upbringing and her tendency to flirt, but lat

In [15]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm, chain_type="stuff", retriever=retriever)
qa_chain({"query": question})

{'query': 'How are the main characters of the book?',
 'result': 'Based on the given context, it is difficult to determine the specific main characters of the book. However, there are several individuals mentioned, such as Adrian, the professor, the captain, and the two girls. Without more information, it is unclear who the main characters are and what their characteristics are.'}